# Initialize project

In [1]:
import deeplabcut
from dlchelperclass import DlcHelperClass as dhc
# Enter your project path
project_path = "/local/data2/LIA_LIU_PONTUS/LIA_LIU/res_50_test-conv_vid-2024-11-04"
# Creates path for config file and videos
config_path, video_path = dhc.get_config_and_video_paths(project_path=project_path)

Loading DLC 3.0.0rc5...


# Extract frames

In [ ]:
# Extract frames for labeling, amount of images and train/test ratio defined in config.yaml
deeplabcut.extract_frames(config_path=config_path, video_path=video_path, mode='automatic')

# Use napari for labeling 

# Creates CSV and h5 files needed for labeling

In [ ]:
deeplabcut.create_training_dataset(config=config_path)

# Train network

In [ ]:
deeplabcut.train_network(config_path, maxiters=1_000_000)

# Creates training_stats.csv

In [ ]:
# Needed for plotting losses
deeplabcut.evaluate_network(config_path, plotting=True)

# Makes prediction on videos

In [ ]:
deeplabcut.analyze_videos(config_path, videos=video_path)

# Create new videos with the models predicted labels

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)

# Evaluate models

In [2]:
# Creates likelihood.txt file and loss/vall_loss plot
dhc.save_mean_likelihood_to_file(project_path=project_path)
dhc.plot_loss_to_png(project_path)

Mean likelihoods saved to /local/data2/LIA_LIU_PONTUS/LIA_LIU/res_50_test-conv_vid-2024-11-04/avg_likelihood.txt
Training and validation loss plot saved to /local/data2/LIA_LIU_PONTUS/LIA_LIU/res_50_test-conv_vid-2024-11-04/training_validation_loss.png


# Extract outlier frames and re-labeling

In [ ]:
deeplabcut.extract_outlier_frames(config=config_path,
                                   shuffle=1,
                                  automatic=True,videos=video_path)

In [ ]:
deeplabcut.merge_datasets(config=config_path)